## Работа с YandexGPT и OpenSearch с помощью фреймворка Langchain

### 1. Устанавливаем необходимые библиотеки

In [ ]:
%pip install -r requirements.txt

После установки библиотек необходимо перегазгрузить Kernel. Для этого в верхнем меню выберите Kernel -> Restart Kernel

### 2. Подключение к кластеру OpenSearch

Документация по подключению: https://cloud.yandex.ru/docs/managed-opensearch/operations/connect

#### Получение SSL-сертификата

In [ ]:
!mkdir -p /home/jupyter/datasphere/project/.opensearch && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
     --output-document /home/jupyter/datasphere/project/.opensearch/root.crt && \
chmod 0600 /home/jupyter/datasphere/project/.opensearch/root.crt

#### Тестируем подключение к кластеру

In [ ]:
CA = '/home/jupyter/datasphere/project/.opensearch/root.crt'

#Укажите пароль для пользователя admin
PASS = '<admin_password>'
#Укажите список хостов кластера
HOSTS = [
   '<адрес_хоста>',
]

In [ ]:
from opensearchpy import OpenSearch

conn = OpenSearch(
  HOSTS,
  http_auth=('admin', PASS),
  use_ssl=True,
  verify_certs=True,
  ca_certs=CA)

print(conn.info())

При правильной настройке подключения код выше вернет объект dict с информацией об имени кластера, uuid, версии и т.д.

### 2. Получаем IAM-токен для работы с YandexGPT

In [ ]:
import time
import jwt
import requests

Для того, чтобы обратиться из DataSphere к YandexGPT в настройках проекта необходимо указать сервисный аккаунт, у которого есть роль ai.languageModels.user. Для сервисного аккаунта необходимо создать **авторизованный** ключ доступа. 

In [1]:
service_account_id = "<sa_id>"
key_id = "<sa_key_id>"
private_key = """
<sa_private_key>
"""

In [ ]:
# Получаем IAM-токен

now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  
                  headers={'Content-Type': 'application/json'},
                  json = {'jwt': encoded_token}).json()
token = x['iamToken']
print(token)

### 3. Создаем индекс в OpenSearch для поиска релевантных документов

Устанавливаем LangChain. **LangChain** — это фреймворк для разработки приложений на базе языковых моделей.

In [ ]:
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.document_loaders import TextLoader
from langchain.chains import LLMChain

from langchain_community.llms import YandexGPT
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

In [ ]:
# Указываем путь к папке с документами в Object Storage. Для этого выберите Сopy Path
source_dir = "/home/jupyter/datasphere/project/yandexgpt-qa-scenarios/data"

In [ ]:
# Создаем объект для считывания документов из бакета в S3
loader = langchain.document_loaders.DirectoryLoader(source_dir, 
                                                    glob="*.txt",
                                                    silent_errors=True,
                                                    show_progress=True, 
                                                    recursive=True)

In [ ]:
# Указываем длины фрагмента, на который разбиваются документы

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [ ]:
# Считываем документы и разбиваем на фрагменты

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP
)
docs = text_splitter.split_documents(documents)

In [ ]:
# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexGPTEmbeddings(iam_token=token, model_uri="emb://<folder_id>/text-search-doc/latest")

In [ ]:
# Строим вектора по документам и добавляем их в базу OpenSearch
docsearch = OpenSearchVectorSearch.from_documents(
     docs,
     embeddings,
     opensearch_url=HOSTS[0],
     http_auth=("admin", PASS),
     use_ssl = True,
     verify_certs = True,
     ca_certs = CA,
     engine = 'lucene'
)

In [ ]:
# Тестируем сохраненные вектора
query = "Как запустить вычисления в DataSphere?"
docs = docsearch.similarity_search(query, k=2)

In [ ]:
docs

### 4. Обращаемся к LLM с выбранными документами

In [ ]:
llm = YandexGPT(iam_token = token,
                model_uri="gpt://<folder_id>/yandexgpt-lite/latest")

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
# Промпт для обработки документов
document_prompt = PromptTemplate(
    input_variables=["page_content"], 
    template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
    Представь себе, что ты сотрудник Yandex Cloud. Твоя задача - вежливо и по мере своих сил отвечать на все вопросы собеседника.
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""
prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context", "query"]
)

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=llm, 
                                      prompt=prompt)
chain = langchain.chains.combine_documents.stuff.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

In [ ]:
#chain.invoke(docs, query=query)
chain.invoke({'query': query, 
              'input_documents': docs})